In [1]:
import json
import os
import fnmatch
import pandas as pd
import re
import numpy as np

In [2]:
####### example to access text of json file


s1 = pd.Series([0,'test', 0])
df = pd.DataFrame([list(s1)], columns=['id', 'text','label'])
index = 0

for path, dirs, files in os.walk('twitter-english/'):
    if (path.endswith('source-tweet') or path.endswith('replies')):
        for filename in os.listdir(path):
            if filename.endswith('.json'):
                with open(os.path.join(path,filename)) as jsonFile:
                    jsonSourceTweet = json.load(jsonFile)

                    text_tweet = jsonSourceTweet['text']
                    id_tweet = int(re.sub("[^0-9]", "", filename))

                    newline = [id_tweet, text_tweet, index]
                    df.loc[index] = newline

                    index += 1




In [3]:
df.head()

,id,text,label
0,529661510369284096,"@mikeyerxa EXCUSE ME, WHERE'S MY INVITE (to th...",0
1,529661727940431872,@halihamilton @mikeyerxa gonna try to make it ...,1
2,529661317087768576,@mikeyerxa you should send Prince a set of ten...,2
3,529666750330449920,"@NobleRobel Girl, don't be mean. Age ain't n...",3
4,529660296080916480,Prince is playing a secret show in Toronto ton...,4


In [4]:
df_label = pd.read_json('train-key.json')

In [9]:
df_label.columns = ['label', 'b']
df_label.head()

,label,b
501760642928635904,comment,NaN
500270212198174720,comment,NaN
524971210275565568,comment,NaN
552836882770690049,comment,NaN
500289931097296897,support,NaN


In [10]:
for i in range(len(df)):
    try:
        index = df.loc[i].id
        df.loc[i, 'label'] = df_label.loc[str(index)].label
    except KeyError:
        df.loc[i, 'label'] = 'not_found'


In [11]:
df.label.value_counts()

comment      2907
not_found    1049
support       910
query         358
deny          344
Name: label, dtype: int64

In [14]:
#save not found labels because they are the eval data
index_to_save = df[df.label == "not_found"].index
mask_save = df.index.isin(index_to_save)
df_eval = df[mask_save]
df_eval.describe()
df_eval.to_csv()

,id
count,1.049000e+03
mean,6.570413e+17
std,1.153509e+17
min,4.982937e+17
25%,5.442976e+17
50%,7.582940e+17
75%,7.677499e+17
max,8.016844e+17


In [16]:
#df_eval.to_csv('eval_data.csv')

In [9]:

#get rid of not found tag

index_to_delete = df[df.label == "not_found"].index
mask = np.logical_not(df.index.isin(index_to_delete))

df_final = df[mask]
df_final.label.value_counts()


comment    2907
support     910
query       358
deny        344
Name: label, dtype: int64

In [21]:
def create_data(path_training, path_answer):
    s1 = pd.Series([0,'test', 0])
    df = pd.DataFrame([list(s1)], columns=['id', 'text','label'])
    index = 0

    for path, dirs, files in os.walk(path_training):
        if (path.endswith('source-tweet') or path.endswith('replies')):
            for filename in os.listdir(path):
                if filename.endswith('.json'):
                    with open(os.path.join(path,filename)) as jsonFile:
                        jsonSourceTweet = json.load(jsonFile)

                        text_tweet = jsonSourceTweet['text']
                        id_tweet = int(re.sub("[^0-9]", "", filename))

                        newline = [id_tweet, text_tweet, index]
                        df.loc[index] = newline

                        index += 1

    #rename column into label                    
    df_label = pd.read_json(path_answer)
    first_col = df_label.columns[0]
    df_label.rename(columns={first_col:'label'}, inplace=True)
    
    #add labels
    for i in range(len(df)):
        try:
            index = df.loc[i].id
            df.loc[i, 'label'] = df_label.loc[str(index)].label
        except KeyError:
            df.loc[i, 'label'] = 'not_found'

    #delete unecessary labels
    index_to_delete = df[df.label == "not_found"].index
    mask = np.logical_not(df.index.isin(index_to_delete))
    df_final = df[mask]


    return df_final

tes_data_csv = create_data('twitter-en-test-data', 'final-eval-key.json')


In [24]:
tes_data_csv.label.value_counts()

comment    771
support    141
deny        92
query       62
Name: label, dtype: int64

ModuleNotFoundError: No module named 'transformers'